In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Assigment4').getOrCreate()

In [2]:
# Load the new hotels file
base_df = spark.read.csv('Hotels_data_Changed.csv',inferSchema=True,header=True)

In [3]:
base_df.show(2)

+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+------------+-------------+-------+
|Snapshot ID|      Snapshot Date|       Checkin Date|Days|Original Price|Discount Price|Discount Code|Available Rooms|          Hotel Name|Hotel Stars|WeekDay|DiscountDiff| DiscountPerc|DayDiff|
+-----------+-------------------+-------------------+----+--------------+--------------+-------------+---------------+--------------------+-----------+-------+------------+-------------+-------+
|          1|2015-07-17 00:00:00|2015-08-12 00:00:00|   5|          1178|          1040|            1|              6|Best Western Plus...|          3|    Wed|         138| 11.714770798|     26|
|          1|2015-07-17 00:00:00|2015-08-19 00:00:00|   5|          1113|           982|            1|              8|Best Western Plus...|          3|    Wed|         131|11.7699910153|     33|
|          1|2015-07-17 0

In [4]:
from pyspark.sql.functions import col

# Get 150 Hotels that have the most rows in data
tophotels = base_df.groupBy("Hotel Name").count().sort(col("count").desc()).head(150)
tophotels

[Row(Hotel Name='Newark Liberty International Airport Marriott', count=5346),
 Row(Hotel Name='Hilton Garden Inn Times Square', count=4892),
 Row(Hotel Name='Residence Inn Newark Elizabeth Liberty International Airport', count=4314),
 Row(Hotel Name='Westin New York at Times Square', count=3792),
 Row(Hotel Name='Loews Regency New York Hotel', count=3617),
 Row(Hotel Name='Viceroy New York', count=3565),
 Row(Hotel Name='Four Seasons Hotel New York', count=3243),
 Row(Hotel Name='Langham Place New York Fifth Avenue', count=3203),
 Row(Hotel Name='The Carlyle A Rosewood Hotel', count=3078),
 Row(Hotel Name='DoubleTree by Hilton Metropolitan - New York City', count=2866),
 Row(Hotel Name='Magnuson Convention Center Hotel', count=2862),
 Row(Hotel Name='Hilton Garden Inn New York West 35th Street', count=2822),
 Row(Hotel Name='Hilton Garden Inn New York-Times Square Central', count=2772),
 Row(Hotel Name='Conrad New York', count=2677),
 Row(Hotel Name='Wyndham Garden Brooklyn Sunset Park

In [5]:
# convert the hotel name + count list to dataframe
top_hotel_names_df = spark.sparkContext.parallelize(tophotels).toDF(['hotel_name','COUNT'])

In [6]:
# Get 40 Checkin dates that have the most rows in data
topdates = base_df.groupBy("Checkin Date").count().sort(col("count").desc()).head(40)
topdates

[Row(Checkin Date=datetime.datetime(2015, 11, 11, 0, 0), count=2403),
 Row(Checkin Date=datetime.datetime(2015, 10, 28, 0, 0), count=2087),
 Row(Checkin Date=datetime.datetime(2015, 11, 6, 0, 0), count=2026),
 Row(Checkin Date=datetime.datetime(2015, 10, 14, 0, 0), count=2017),
 Row(Checkin Date=datetime.datetime(2015, 11, 4, 0, 0), count=2014),
 Row(Checkin Date=datetime.datetime(2015, 10, 21, 0, 0), count=1979),
 Row(Checkin Date=datetime.datetime(2015, 8, 19, 0, 0), count=1954),
 Row(Checkin Date=datetime.datetime(2015, 11, 5, 0, 0), count=1868),
 Row(Checkin Date=datetime.datetime(2015, 10, 29, 0, 0), count=1821),
 Row(Checkin Date=datetime.datetime(2015, 8, 12, 0, 0), count=1818),
 Row(Checkin Date=datetime.datetime(2015, 10, 22, 0, 0), count=1748),
 Row(Checkin Date=datetime.datetime(2015, 11, 12, 0, 0), count=1732),
 Row(Checkin Date=datetime.datetime(2015, 9, 10, 0, 0), count=1729),
 Row(Checkin Date=datetime.datetime(2015, 10, 30, 0, 0), count=1728),
 Row(Checkin Date=datetime

In [7]:
# convert the name + count list to dataframe
top_checkin_dates_df = spark.sparkContext.parallelize(topdates).toDF(['checkin_date','COUNT'])

In [8]:
only_hotel_names_df = top_hotel_names_df.drop("COUNT")
only_checkin_dates_df = top_checkin_dates_df.drop("COUNT")
joint_df = only_hotel_names_df.crossJoin(only_checkin_dates_df)

In [9]:
price_codes_df = spark.range(1,5).withColumnRenamed("id","price_code")
joint_with_price_codes_df = joint_df.crossJoin(price_codes_df)

In [10]:
joint_with_price_codes_df.show(5)

+--------------------+-------------------+----------+
|          hotel_name|       checkin_date|price_code|
+--------------------+-------------------+----------+
|Newark Liberty In...|2015-11-11 00:00:00|         1|
|Newark Liberty In...|2015-11-11 00:00:00|         2|
|Newark Liberty In...|2015-11-11 00:00:00|         3|
|Newark Liberty In...|2015-11-11 00:00:00|         4|
|Newark Liberty In...|2015-10-28 00:00:00|         1|
+--------------------+-------------------+----------+
only showing top 5 rows



In [11]:
#dummy_prices_df = spark.range(-1,0).withColumnRenamed("id","price")
#complete_df = joint__with_price_codes_df.crossJoin(dummy_prices_df)

In [12]:
temp_dict = {}
for line in joint_with_price_codes_df.collect():
    temp_dict[line.hotel_name, line.checkin_date, line.price_code] = -1

In [15]:
grouped_df = base_df.groupBy("Hotel Name", "Checkin Date", "Discount Code").min("Discount Price")
grouped_renamed_df = grouped_df.withColumnRenamed("Hotel Name", "hn").withColumnRenamed("Checkin Date", "cd").withColumnRenamed("Discount Code", "dc").withColumnRenamed("min(Discount Price)","dp")
grouped_renamed_df.show()

+--------------------+-------------------+---+----+
|                  hn|                 cd| dc|  dp|
+--------------------+-------------------+---+----+
| Park Hyatt New York|2015-07-31 00:00:00|  3|3455|
|New York Marriott...|2015-08-06 00:00:00|  4|1670|
|Hilton Garden Inn...|2015-08-13 00:00:00|  4|1007|
|Gild Hall A Thomp...|2015-09-09 00:00:00|  1|1396|
|Omni Berkshire Place|2015-08-27 00:00:00|  2|1156|
|Westin New York a...|2015-08-22 00:00:00|  3|1265|
|Hyatt Union Squar...|2015-09-06 00:00:00|  3|2085|
| The Kitano New York|2015-09-19 00:00:00|  4|2075|
|Andaz Wall Street...|2015-08-25 00:00:00|  3|1365|
|Hilton Newark Air...|2015-08-21 00:00:00|  1| 710|
|Hyatt Times Squar...|2015-09-18 00:00:00|  2|1785|
|Hilton Garden Inn...|2015-09-27 00:00:00|  4|1836|
|Hilton Newark Air...|2015-08-07 00:00:00|  1| 710|
|Hilton Garden Inn...|2015-08-04 00:00:00|  1|1755|
|Smyth A Thompson ...|2015-09-30 00:00:00|  3|2525|
|Omni Berkshire Place|2015-09-24 00:00:00|  3|3735|
|   The Milb

In [16]:
for row in grouped_renamed_df.collect():
    temp_dict[row.hn, row.cd, row.dc] = row.dp

In [17]:
for i in range(1,200):
    print (temp_dict[tuple(list(temp_dict)[i])])

906
1008
-1
788
856
966
-1
1315
1195
-1
-1
804
863
965
-1
796
864
966
-1
1195
1185
1134
1144
1515
1588
1515
1525
1082
1111
1084
-1
1715
1645
1535
1662
1195
1235
-1
1149
688
717
769
-1
749
771
835
-1
-1
-1
-1
-1
871
-1
1070
-1
696
687
739
-1
1532
1586
1631
1681
1606
1674
1719
-1
-1
1565
1516
1728
1356
1426
1475
1965
1336
1396
1445
-1
2276
2276
2316
2316
3095
2476
2436
2476
2196
2316
2795
2276
3295
3015
3085
3345
2195
2436
2316
2516
-1
1625
1705
1999
2035
2125
2215
2197
2165
1939
1859
-1
2205
2125
2245
1984
1835
1885
1939
1885
-1
-1
5235
5355
4995
4815
4675
4775
5355
4875
5335
-1
-1
4575
4675
4775
4995
4875
5235
4775
2923
2888
3028
3168
2993
3028
3168
-1
-1
-1
-1
-1
3434
3431
3671
3833
3273
3368
3606
3817
3285
3420
3555
3690
4560
4740
4860
-1
-1
-1
-1
-1
-1
3780
3870
3960
-1
-1
-1
-1
917
1020
987
1133
1565
-1
1545
1465
1163
916
983
-1
1445
1405
1685
1335
-1
983
1057
1061


In [29]:
rowsList = []
for key in list(temp_dict.keys()):
    rowsList.append([key[0], key[1], key[2] ,temp_dict[key]])

In [32]:
final_df = spark.sparkContext.parallelize(rowsList).toDF(["hotel_name", "checkin_date", "discount_code", "discount_price"])

In [38]:
final_df.filter("discount_price<261").filter("discount_price>1").show()

+-------------+-------------------+-------------+--------------+
|   hotel_name|       checkin_date|discount_code|discount_price|
+-------------+-------------------+-------------+--------------+
|The SunBright|2015-11-30 00:00:00|            4|           260|
|The SunBright|2015-11-30 00:00:00|            2|           260|
+-------------+-------------------+-------------+--------------+



In [51]:
min_price = final_df.filter("discount_price>-1").groupBy().min("discount_price").head()[0]
max_price = final_df.filter("discount_price>-1").groupBy().max("discount_price").head()[0]

In [69]:
from pyspark.sql import functions as F
normalized_df = final_df.select(final_df[0], final_df[1], final_df[2], final_df[3], F.when(final_df.discount_price > -1, 100 * (final_df.discount_price - min_price) / (max_price - min_price)).otherwise(-1).alias("normalized_price"))

In [70]:
normalized_df.show()

+--------------------+-------------------+-------------+--------------+------------------+
|          hotel_name|       checkin_date|discount_code|discount_price|  normalized_price|
+--------------------+-------------------+-------------+--------------+------------------+
|Newark Liberty In...|2015-11-11 00:00:00|            1|           838|3.4323040380047507|
|Newark Liberty In...|2015-11-11 00:00:00|            2|           906| 3.836104513064133|
|Newark Liberty In...|2015-11-11 00:00:00|            3|          1008| 4.441805225653207|
|Newark Liberty In...|2015-11-11 00:00:00|            4|            -1|              -1.0|
|Newark Liberty In...|2015-10-28 00:00:00|            1|           788|3.1353919239904986|
|Newark Liberty In...|2015-10-28 00:00:00|            2|           856| 3.539192399049881|
|Newark Liberty In...|2015-10-28 00:00:00|            3|           966| 4.192399049881235|
|Newark Liberty In...|2015-10-28 00:00:00|            4|            -1|              -1.0|